3.1 텐서플로란?

3.2 케라스란?

3.3 케라스와 텐서플로의 간략한 역사

3.4 딥러닝 작업 환경 설정하기

3.5 텐서플로 시작하기

3.6 신경망의 구조 : 핵심 Keras API 이해하기

3.7 요약

3.1 텐서플로란?
- 구글에서 만든 파이썬 기반의 무료 오픈 소스 머신 러닝 플랫폼
- 넘파이와 매우 비슷하게 텐서플로의 핵심 목적은 엔지니어와 연구자가 수치 텐서에 대한 수학적 표현을 적용할 수 있도록 하는 것
- 하지만, 텐서플로는 다음과 같은 넘파이의 기능을 넘어선다.
    
    - 미분 가능한 어떤 표현식에 대해서도 자동으로 그레이디언트를 계산할 수 있으므로 머신 러닝에 매우 적합
    - CPU 뿐만 아니라 고도로 병렬화된 하드웨어 가속기은 GPU, TPU에서도 실행 o
    - 텐서플로에서 정의한 계산은 여러 머신에 쉽게 분산시킬 수 o
    - 텐서플로 프로그램은 C++, 자바스크립트 .. 다른런타임에 맞게 변환 할 수 o => 따라서 텐서플로 애플리케이션을 실전 환경에 쉽게 배포 가능

3.2 케라스란?
- 텐서플로 위에 구축된 파이썬용 딥러닝 API로 어떤 종류의 딥러닝 모델도 쉽게 만들고 훈련할 수 있는 방법 제공

3.3 케라스와 텐서플로의 간략한 역사
- 케라스는 텐서플로보다 먼저 나왔다. 케라스가 텐서플로 위에 구축된 것인데 어떻게 텐서플로보다 먼저 나왔을까?

    ㄴ 케라스는 원래 씨아노를 위한 라이브러리 였음
    
    
- 텐서플로가 기술적으로 성숙한 수준에 도달한 후 케라스의 기본 백엔드가 되었다. 그후 케라스에 새로운 백엔드 옵션이 추가되었지만(CNTK, MXNet), 현재까지는 케라스는 텐서플로를 사용하는 단일 백엔드 API가 되었다.

3.4 딥러닝 작업 환경 설정
1. NVIDIA GPU 구입하기
2. 구글 클라우드나 AWS EC2의 GPU 인스턴스 사용
3. 구글이 제공하는 노트북 서비스인 코랩의 무료 GPU 런타임 사용하기 => 우린 주로 이걸 사용..

### 주피터 노트북 : 권장하는 딥러닝 실험 도구
### 코랩 사용하기

**- 코랩 시작하기**

**- pip로 패키지 설치하기**

**- GPU 런타임 사용하기**

3.5 텐서플로 시작하기

### 1. 상수 텐서와 변수
모두 1인 텐서

```
import tensorflow as tf
x = tf.ones(shape=(2, 1))
print(x)
```
모두 0인 텐서

```
x = tf.zeros(shape=(2, 1))
print(x)

```
랜덤 텐서
```
x = tf.random.normal(shape=(3, 1), mean=0., stddev=1.)
print(x)
```
넘파이 배열에 값 할당
```
import numpy as np
x = np.ones(shape=(2, 2))
x[0, 0] = 0.
```
넘파이와 달리 텐서플로는 텐서에 값을 할당하지 못한다.
```
x = tf.ones(shape=(2,2))
x[0,0] = 0.
```
ㄴ이렇게 하면 오류 발생

해결방안 : 텐서플로 변수 만들기 (assign 사용)

```
v = tf.Variable(initial_value=tf.random.normal(shape=(3, 1)))
print(v)
```
텐서플로 변수에 값 할당
```
v.assign(tf.ones((3, 1)))
```
변수 일부에 값 할당하기
```
v[0, 0].assign(3.)
```
비슷하게 assign_add()와 assign_sub() 사용하기
```
v.assign_add(tf.ones((3, 1)))
```
### 2. 텐서 연산 : 텐서플로에서 수학 계산하기
기본적인 수학 연산
```
a = tf.ones((2, 2))
b = tf.square(a)
c = tf.sqrt(a)
d = b + c
e = tf.matmul(a, b)
e *= d
```
### 3. GradientTape API 다시 살펴보기
GradientTape 사용하기(입력 텐서가 텐서플로 변수인 경우)
```
input_var = tf.Variable(initial_value=3.)
with tf.GradientTape() as tape:
   result = tf.square(input_var)
gradient = tape.gradient(result, input_var)
```
상수 입력 텐서와 함께 GradientTape 사용하기 (입력 텐서가 상수 텐서의 경우tape.watch() 호출하기)
```
input_const = tf.constant(3.)
with tf.GradientTape() as tape:
   tape.watch(input_const)
   result = tf.square(input_const)
gradient = tape.gradient(result, input_const)
```
그레이디언트 테이프를 중첩하여 이계도 그레이디언트를 계산하기 (like, 미분에 미분)
```
time = tf.Variable(0.)
with tf.GradientTape() as outer_tape:
    with tf.GradientTape() as inner_tape:
        position =  4.9 * time ** 2
    speed = inner_tape.gradient(position, time)
acceleration = outer_tape.gradient(speed, time)
```
### 4. 엔드-투-엔드 예제 : 텐서플로 선형 분류기
선형 분류기의 변수 만들기
```
input_dim = 2
output_dim = 1 # (0에 가까우면 class 0으로 아니면 1로 .. 이렇게 1차원적)
W = tf.Variable(initial_value=tf.random.uniform(shape=(input_dim, output_dim)))
b = tf.Variable(initial_value=tf.zeros(shape=(output_dim,)))
```
정방향 패스 함수
```
def model(inputs):
    return tf.matmul(inputs, W) + b
```
평균 제곱 오차 손실 함수
```
def square_loss(targets, predictions):
    per_sample_losses = tf.square(targets - predictions)
    return tf.reduce_mean(per_sample_losses)
```
훈련 스텝 함수
```
learning_rate = 0.1

def training_step(inputs, targets):
    with tf.GradientTape() as tape:
        predictions = model(inputs)
        loss = square_loss(targets, predictions)
    grad_loss_wrt_W, grad_loss_wrt_b = tape.gradient(loss, [W, b])
    W.assign_sub(grad_loss_wrt_W * learning_rate)
    b.assign_sub(grad_loss_wrt_b * learning_rate)
    return loss
```
배치 훈련 루프
```
for step in range(40):
    loss = training_step(inputs, targets)
    print(f"{step}번째 스텝의 손실: {loss:.4f}")
```
그래프 그려보기
```
predictions = model(inputs)
plt.scatter(inputs[:, 0], inputs[:, 1], c=predictions[:, 0] > 0.5)
plt.show()
```
선형 분류된 직선 포함한 그래프 그려보기
```
x = np.linspace(-1, 4, 100)
# 사실 100개의 x 축 좌표를 만들 필요 없이 시작과 종료 위치만 있어도 됩니다.
# x = [-1, 4]
y = - W[0] /  W[1] * x + (0.5 - b) / W[1]
plt.plot(x, y, "-r")
plt.scatter(inputs[:, 0], inputs[:, 1], c=predictions[:, 0] > 0.5)
plt.show()
```

3.6 신경망의 구조: 핵심 Keras API 이해하기
### 1. 층: 딥러닝의 구성 요소
Layer의 서브클래스로 구현한 Dense 층
```
from tensorflow import keras

class SimpleDense(keras.layers.Layer):

    def __init__(self, units, activation=None):
        super().__init__()
        self.units = units
        self.activation = activation

    def build(self, input_shape):
        input_dim = input_shape[-1]
        self.W = self.add_weight(shape=(input_dim, self.units),
                                 initializer="random_normal")
        self.b = self.add_weight(shape=(self.units,),
                                 initializer="zeros")

    def call(self, inputs):
        y = tf.matmul(inputs, self.W) + self.b
        if self.activation is not None:
            y = self.activation(y)
        return y
```
사용해 보면,
```
my_dense = SimpleDense(units=32, activation=tf.nn.relu)
input_tensor = tf.ones(shape=(2, 784))
output_tensor = my_dense(input_tensor)
print(output_tensor.shape)
```
(2, 32)

자동 크기 추론: 동적으로 층 만들기
```
from tensorflow.keras import layers
layer = layers.Dense(32, activation="relu")
```
케라스의 경우 크기 호환성 걱정 필요 x 동적으로 할당해줌
```
from tensorflow.keras import models
from tensorflow.keras import layers
model = models.Sequential([
    layers.Dense(32, activation="relu"),
    layers.Dense(32)
])
```
예)
```
model = keras.Sequential([
    SimpleDense(32, activation="relu"),
    SimpleDense(64, activation="relu"),
    SimpleDense(32, activation="relu"),
    SimpleDense(10, activation="softmax")
])
```
### 2. 층에서 모델로
모델 구조 정의
### 3. “컴파일” 단계: 학습 과정 설정
손실 함수, 옵티마이저, 측정 지표 선택해줘야함
```
model = keras.Sequential([keras.layers.Dense(1)]) # 모델 구조 정의
model.compile(optimizer="rmsprop",
              loss="mean_squared_error",
              metrics=["accuracy"])
```
다른 방법으로는
```
model.compile(optimizer=keras.optimizers.RMSprop(),
              loss=keras.losses.MeanSquaredError(),
              metrics=[keras.metrics.BinaryAccuracy()])
```
옵티마이저
- SGD(모멘텀 선택 가능)
- RMSprop
- Adam
- Adagrad
손실 함수
- CategoricalCrossentropy
- SparseCategoricalCrossentropy
- BinaryCrossentropy
- MeanSquaredError
- KLDivergence
- CosineSimilarity
측정 지표
- CategoricalAccuracy
- SparseCategoricalAccuracy
- BinaryAccuracy
- AUC
- Precision
- Recall
### 4. 손실 함수 선택하기
BinaryCrossentropy => **2개의 클래스가 있는 분류**
CategoricalCrossentropy => **여러 개의 클래스가 있는 분류**

### 5. fit() 메서드 이해하기
넘파이 데이터로 fit() 메서드 호출하기
```
history = model.fit(
    inputs,
    targets,
    epochs=5,
    batch_size=128
)
```
Epoch 1/5
16/16 [==============================] - 1s 2ms/step - loss: 10.1885 - binary_accuracy: 0.0020
Epoch 2/5
16/16 [==============================] - 0s 2ms/step - loss: 9.8145 - binary_accuracy: 0.0020 
Epoch 3/5
16/16 [==============================] - 0s 2ms/step - loss: 9.4990 - binary_accuracy: 0.0020
Epoch 4/5
16/16 [==============================] - 0s 2ms/step - loss: 9.1918 - binary_accuracy: 0.0020
Epoch 5/5
16/16 [==============================] - 0s 1ms/step - loss: 8.8920 - binary_accuracy: 0.0020
```
history.history
```
{'loss': [10.188525199890137,
  9.814451217651367,
  9.498977661132812,
  9.191787719726562,
  8.891997337341309],
 'binary_accuracy': [0.0020000000949949026,
  0.0020000000949949026,
  0.0020000000949949026,
  0.0020000000949949026,
  0.0020000000949949026]}
### 6. 검증 데이터에서 손실과 측정 지표 모니터링하기
validation_data 매개변수 사용하기

**validation_data=(val_inputs, val_targets)** 사용하기
```
model = keras.Sequential([keras.layers.Dense(1)])
model.compile(optimizer=keras.optimizers.RMSprop(learning_rate=0.1),
              loss=keras.losses.MeanSquaredError(),
              metrics=[keras.metrics.BinaryAccuracy()])

indices_permutation = np.random.permutation(len(inputs))
shuffled_inputs = inputs[indices_permutation]
shuffled_targets = targets[indices_permutation]

num_validation_samples = int(0.3 * len(inputs))
val_inputs = shuffled_inputs[:num_validation_samples]
val_targets = shuffled_targets[:num_validation_samples]
training_inputs = shuffled_inputs[num_validation_samples:]
training_targets = shuffled_targets[num_validation_samples:]
model.fit(
    training_inputs,
    training_targets,
    epochs=5,
    batch_size=16,
    validation_data=(val_inputs, val_targets)
)
```
Epoch 1/5
88/88 [==============================] - 1s 4ms/step - loss: 0.2450 - binary_accuracy: 0.9093 - val_loss: 0.0913 - val_binary_accuracy: 0.9850
Epoch 2/5
88/88 [==============================] - 0s 3ms/step - loss: 0.0680 - binary_accuracy: 0.9643 - val_loss: 0.0270 - val_binary_accuracy: 1.0000
Epoch 3/5
88/88 [==============================] - 0s 5ms/step - loss: 0.0779 - binary_accuracy: 0.9507 - val_loss: 0.0232 - val_binary_accuracy: 0.9967
Epoch 4/5
88/88 [==============================] - 0s 4ms/step - loss: 0.0765 - binary_accuracy: 0.9407 - val_loss: 0.0598 - val_binary_accuracy: 0.9883
Epoch 5/5
88/88 [==============================] - 0s 5ms/step - loss: 0.0638 - binary_accuracy: 0.9721 - val_loss: 0.0726 - val_binary_accuracy: 0.9533
<keras.callbacks.History at 0x7f0787cb8bd0>
### 7. 추론: 훈련한 모델 사용하기
```
predictions = model(new_inputs)
```
이건 call 메서드 호출하는 것

=> 데이터 한 번에 처리하는 것 => 좋은 방법은 아님

**predict()메서드를 사용하는 것이!**

```
predictions = model.predict(val_inputs, batch_size=128)
print(predictions[:10])
```
5/5 [==============================] - 0s 3ms/step
[[0.2212547 ]
 [0.46757108]
 [1.3064125 ]
 [0.36760795]
 [0.98762196]
 [1.2589831 ]
 [0.9364893 ]
 [0.2468108 ]
 [0.94096804]
 [0.4054407 ]]